In [1]:
# Dependencies
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
There is no [win32] chromedriver for browser 92.0.4515 in cache
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.107/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\helena\.wdm\drivers\chromedriver\win32\92.0.4515.107]


In [3]:
# URL of page to be scraped
# url = 'http://house.speakingsame.com/p.php?q=Perth%2C+WA'
url = 'http://house.speakingsame.com/p.php?q=Subiaco%2C+WA'
browser.visit(url)

In [4]:
addresses = []
properties_info = []

pages = np.arange(0, 30, 1)

for page in pages:
    
#     page_url = "http://house.speakingsame.com/p.php?q=Perth&p="+ str(page) +"&s=1&st=&type=&count=300&region=Perth&lat=0&lng=0&sta=wa&htype=&agent=0&minprice=0&maxprice=0&minbed=0&maxbed=0&minland=0&maxland=0"
    page_url = "http://house.speakingsame.com/p.php?q=Subiaco&p="+ str(page) +"&s=1&st=&type=&count=300&region=Subiaco&lat=0&lng=0&sta=wa&htype=&agent=0&minprice=0&maxprice=0&minbed=0&maxbed=0&minland=0&maxland=0"
    
    browser.visit(page_url)
   
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    addr_spans = soup.find_all('span', class_='addr')

    for span in addr_spans:
        addresses.append(span.a.text)
    addresses

    tables = soup.find_all('table', style="font-size:13px") #tables list (except 0,1 index) contians list of properties in the page

    for i in range(2,12):
        try:
            info = []
            trs = tables[i].find_all('tr')
            for tr in trs:
                info.append(tr.td.text)
        except:
            info.append('')
        properties_info.append(info)
        
  
    
properties_info  

[['Sold $505,000 in Aug 2021',
  'Rent $450pw in May 2019',
  'Apartment: 1  1  1 ',
  'Land size: 3,437 sqm | Building size: 75 sqm',
  'Agent: Carl McNeill',
  'SOLD above marketing price!. This is one very cool pad! Loving the blackboard wall in this interesting apartment in the middle of the action. Its the perfect bachelor pad! Something really...more',
  'Floorplan | Street view | Nearby'],
 ['Sold $410,000 in Aug 2021',
  'Last Sold $376,387 in Oct 2003',
  'Rent $400pw in May 2021',
  'Unit: 1  1  1 ',
  'Land size: 158 sqm | Building size: 65 sqm',
  'Agent: Sean Roberts',
  'Street view | Nearby'],
 ['Sold $1,710,000 in Aug 2021',
  'House: ',
  'Agent: Kirk Bellerby',
  'Lot/Plan No: P000428 53more',
  'Street view | Nearby'],
 ['Sold $575,000 in Aug 2021',
  'Rent $770pw in Dec 2014',
  'Apartment: 2  2  2 ',
  'Land size: 166 sqm | Building size: 110 sqm',
  'Agent: Cameron McGregor',
  'Street view | Nearby'],
 ['Sold $1,800,000 in Jul 2021',
  'Last Sold $1,245,000 in Ap

In [5]:
len(properties_info)

300

In [6]:
len(addresses)

300

In [7]:
Properties = []
for i in range(0, len(addresses)):
    
    Property_dict = {}
    Property_dict['address'] = addresses[i]
    
    for j in range (0, len(properties_info[i])):
        
        try:
                    
            if properties_info[i][j].split(' ')[0] == "Sold": 
                Property_dict['price'] = properties_info[i][j].split(' ')[1]
                Property_dict['sold date'] = properties_info[i][j].split('in ')[1]

            if properties_info[i][j].split(' ')[0] in ["Apartment:", "House:", "Townhouse:"]:
                Property_dict['type'] = properties_info[i][j].split(' ')[0].split(':')[0]
                Property_dict['bedrooms'] = properties_info[i][j].split(' ')[1]
                try:
                    Property_dict['bathrooms'] = properties_info[i][j].split(' ')[3]
                except:
                    Property_dict['bathrooms'] = ''
                try:
                    Property_dict['car space'] = properties_info[i][j].split(' ')[5]
                except:
                    Property_dict['car space'] = ''
                    
            if properties_info[i][j].split(' ')[0] == "Agent:":
                Property_dict['agent'] = properties_info[i][j].split(': ')[1]
                
            if properties_info[i][j].split(' ')[0] == "Rent":
                Property_dict['rent'] = properties_info[i][j].split(' ')[1]
                Property_dict['rent date'] = properties_info[i][j].split('in ')[1]
                
            if properties_info[i][j].split(' ')[1] == "size:":
                if properties_info[i][j].split(' ')[0] == "Land":
                    Property_dict['Land size'] = properties_info[i][j].split(' ')[2]
                    try:
                        if properties_info[i][j].split(' ')[5] == "Building":
                            Property_dict['Building size'] = properties_info[i][j].split(' ')[7]
                    except:
                        continue
                elif properties_info[i][j].split(' ')[0] == "Building":
                    Property_dict['Building size'] = properties_info[i][j].split(' ')[2]
            
            
        except:
            continue

  
    Properties.append(Property_dict)
        
Properties

[{'address': '9/3 Brigid Road',
  'price': '$505,000',
  'sold date': 'Aug 2021',
  'rent': '$450pw',
  'rent date': 'May 2019',
  'type': 'Apartment',
  'bedrooms': '1',
  'bathrooms': '1',
  'car space': '1',
  'Land size': '3,437',
  'Building size': '75',
  'agent': 'Carl McNeill'},
 {'address': '36/2 Wexford Street',
  'price': '$410,000',
  'sold date': 'Aug 2021',
  'rent': '$400pw',
  'rent date': 'May 2021',
  'Land size': '158',
  'Building size': '65',
  'agent': 'Sean Roberts'},
 {'address': '244 Heytesbury Road',
  'price': '$1,710,000',
  'sold date': 'Aug 2021',
  'type': 'House',
  'bedrooms': '',
  'bathrooms': '',
  'car space': '',
  'agent': 'Kirk Bellerby'},
 {'address': '54/1 Station Street',
  'price': '$575,000',
  'sold date': 'Aug 2021',
  'rent': '$770pw',
  'rent date': 'Dec 2014',
  'type': 'Apartment',
  'bedrooms': '2',
  'bathrooms': '2',
  'car space': '2',
  'Land size': '166',
  'Building size': '110',
  'agent': 'Cameron McGregor'},
 {'address': '59 

In [10]:
len(Properties)

300

In [11]:
Subiaco_df = pd.DataFrame(Properties)

In [12]:
Subiaco_df

,address,price,sold date,rent,rent date,type,bedrooms,bathrooms,car space,Land size,Building size,agent
0,9/3 Brigid Road,"$505,000",Aug 2021,$450pw,May 2019,Apartment,1,1,1,"3,437",75,Carl McNeill
1,36/2 Wexford Street,"$410,000",Aug 2021,$400pw,May 2021,NaN,NaN,NaN,NaN,158,65,Sean Roberts
2,244 Heytesbury Road,"$1,710,000",Aug 2021,NaN,NaN,House,,,,NaN,NaN,Kirk Bellerby
3,54/1 Station Street,"$575,000",Aug 2021,$770pw,Dec 2014,Apartment,2,2,2,166,110,Cameron McGregor
4,59 Heytesbury Road,"$1,800,000",Jul 2021,NaN,NaN,House,3,2,,354,144,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,310/74 Price Street,"$299,000",Apr 2019,NaN,NaN,Apartment,1,1,1,NaN,NaN,NaN
296,23 Heytesbury Road,"$1,675,000",Apr 2019,NaN,NaN,House,4,2,1,NaN,NaN,Space Real Estate - Cottesloe
297,118 Barker Road,"$1,540,000",Apr 2019,NaN,NaN,House,4,2,1,NaN,NaN,NaN
298,11 Union Street,"$1,410,000",Apr 2019,NaN,NaN,House,3,2,1,539,NaN,NaN


In [14]:
Subiaco_df.to_csv("Subiaco_properties.csv")